In [76]:
import pandas as pd
import os
import json
import glob
import sqlite3

In [77]:
with open('../parameters.json') as json_data:
    params = json.load(json_data)

DATA_DIR = params['DIRECTORIES']['DATA_DIR']
TMP_DIR = params['DIRECTORIES']['TMP_DIR']
PRICE_FILE = params['FILES']['PRICE_FILE']
STATION_FILE = params['FILES']['STATION_FILE']
SERVICE_FILE = params['FILES']['SERVICE_FILE']
GAS_DB = os.path.join(DATA_DIR, params['DATABASES']['GAS_DB'])

In [32]:
gas_files = glob.glob(os.path.join(DATA_DIR, "Prix*.zip"))
stations_file = 'Stations2016.zip'
services_file = 'Services2016.zip'

In [38]:
dfall = pd.DataFrame()

for year in gas_files:
    df = pd.read_csv(year,
                     sep=';', header=None, dtype = {1: str}
                    )
    df.columns = ['id_station', 'code_postal', 'type_station',
                  'latitude', 'longitude', 'date_releve',
                  'type_carburant', 'libelle_carburant', 'prix']
    dfall = pd.concat([dfall, df], axis=0)

assert dfall.shape[1] == 9, "Pas le bon nombre de colonnes"

In [56]:
dfstations = pd.read_csv(os.path.join(DATA_DIR, stations_file),
                         sep='|',
                         header=None,
                         dtype={1: str},
                         encoding='utf-8')
dfstations.columns = ['id_station', 'code_postal', 'type_station',
                      'latitude', 'longitude', 'adresse', 'ville']

In [57]:
dfservices = pd.read_csv(os.path.join(DATA_DIR, services_file),
                         sep='|',
                         header=None,
                         dtype={1: str},
                         encoding='utf-8')
dfservices.columns = ['id_station', 'code_postal', 'type_station',
                      'latitude', 'longitude', 'services']

In [72]:
dfall.to_pickle(os.path.join(DATA_DIR, PRICE_FILE))
dfstations.to_pickle(os.path.join(DATA_DIR, STATION_FILE))
dfservices.to_pickle(os.path.join(DATA_DIR, SERVICE_FILE))

In [79]:
cnx = sqlite3.connect(GAS_DB)

In [88]:
dfall.to_sql('prices', cnx, index=False, if_exists='replace')

In [87]:
dfstations.to_sql('stations', cnx, index=False, if_exists='replace')

In [86]:
dfservices.to_sql('services', cnx, index=False, if_exists='replace')